## 任务 1：训练模型

创建机器学习 (ML) 模型的过程从数据处理开始。数据处理完成后，选择一种机器学习算法来训练模型。模型训练的目标是创建一个模型，您可以使用该模型对未来的数据进行预测。处理后的数据必须包含目标，但未来的数据不包含目标（未标记）。算法在训练数据中找到将输入数据属性映射到目标的模式。然后，算法输出捕获这些模式的机器学习模型。有了模型后，您可以对不包含目标值的新数据进行预测。

例如，如果您想训练一个机器学习模型来预测电子邮件是否为垃圾邮件，您可以为您的模型提供训练数据，其中包含您知道目标的电子邮件（本例中是一个标签，用来判断电子邮件是否为垃圾邮件）。使用这些数据，算法将创建一个模型来预测电子邮件是否为垃圾邮件。您可以使用该模型来预测未来的电子邮件标签。

在此任务中，您要预测某人的收入是否少于 50,000 美元。您的模型正在进行自我优化训练，以便它能够尽可能准确地预测某人的收入是否少于 50,000 美元。模型训练需要一些配置，包括要使用哪种算法进行训练。在此任务中，您将使用 XGBoost (eXtreme Gradient Boosting) 算法。当您训练模型时，还需要配置您的超参数。超参数是控制训练作业过程的参数。可以对它们进行调整以更改训练作业中的各个步骤。就模型性能和准确性而言，选择正确的超参数集非常重要。在您训练模型之后，评估模型并查看模型构件。

### 任务 1.1：设置环境

在开始训练模型之前，请安装所有必要依赖项。

In [ ]:
#Install matplotlib and restart kernel
%pip install matplotlib
%pip uninstall bokeh -y
%pip install bokeh==2.4.2
%reset -f

# Install dependencies
import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from sagemaker.debugger import Rule, rule_configs
from IPython.display import FileLink, FileLinks
from sagemaker import image_uris
from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime
from pathlib import Path

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')

注意：如果您在输出中看到错误：“错误：pip 的依赖项解析程序目前不会将已安装的所有软件包考虑在内”，您可以忽略软件包依赖项错误，并继续执行后续步骤。

接下来，导入数据集。在本练习中，假设有人将数据文件从 Amazon SageMaker Data Wrangler 导出到 Amazon Simple Storage Service (Amazon S3) 存储桶中。将数据集拆分为训练数据集 (70%)、验证数据集 (20%) 和测试数据集 (10%)。在训练期间将使用训练数据集和验证数据集。在部署后进行模型评估时将使用测试数据集。

要使用 Amazon SageMaker 进行训练，数据集必须为 libSVM 或 CSV 格式。本实验使用 CSV 格式进行训练。

要查看在上一个实验中创建的数据集文件，请遵循以下步骤：

1.从左侧菜单栏选择存储桶图标。

1.在存储桶列表中，选择名称中包含 **labdatabucket** 的 Amazon S3 存储桶。

打开 .csv 文件将在 SageMaker Studio 中打开新的选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_2.ipynb** 选项卡拖到一边，或者选择 **lab_2.ipynb** 选项卡，然后从工具栏中选择 **File**（文件）和 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览 .csv 文件时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完 .csv 文件后，通过选择 **lab_2.ipynb** 选项卡返回至笔记本。

1.依次选择（双击）**scripts** 文件夹、**data** 文件夹、**train** 文件夹和 **adult_data_processed_train.csv** 文件以查看其内容。

1.在左侧窗格中，从 <i aria-hidden="true" class="fas fa-folder" style="color:white"></i> **/ ... /data/train/** 面包屑导航链接中选择 **data**。

1.依次选择（双击）**validation** 文件夹、**adult_data_processed_validation.csv** 文件以查看其内容。

您已经查看数据集文件。现在，配置训练作业用作输入的训练路径和验证路径。

In [ ]:
# Import the datasets
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)
prefix = 'scripts/data'
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Configure the training paths
train_path = f"s3://{bucket}/{prefix}/train/adult_data_processed_train.csv"
validation_path = f"s3://{bucket}/{prefix}/validation/adult_data_processed_validation.csv"

# Set up the TrainingInput objects
train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

# Print the training and validation paths
print(f'Training path: {train_path}')
print(f'Validation path: {validation_path}')

# Set the container, name, and tags
create_date = strftime("%m%d%H%M")
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')
run_name = 'lab-2-run-{}'.format(create_date)

### 任务 1.2：配置估算器对象

估算器是用于 SageMaker 训练的高级接口。您可以通过提供所需的参数（如 AWS Identity and Access Management (IAM) 角色、计算实例计数和类型以及 Amazon S3 输出路径）来创建估算器对象。本实验将 XGBoost 内置算法用于 SageMaker 通用估算器。XGBoost 是梯度提升树算法的一种常用且高效的开源实现。梯度提升是一种有监督学习算法，它尝试将一组较简单和较弱的模型的一系列估计值结合在一起，从而准确地预测目标变量。XGBoost 算法在处理各种数据类型、关系、分布以及可微调的各种超参数方面表现良好。可以使用 XGBoost 进行回归、分类（二元类和多类）和排序问题。在本例中，您将使用 XGBoost 来解决分类问题（是否有人的收入低于 50,000 美元）。

在本实验中，您将使用 *sagemaker.estimator.Estimator* 类创建一个 XGBoost 估算器。在下面的示例代码中，XGBoost 估算器被命名为 *xgb_model*。要构造 SageMaker 估算器，请指定以下参数：

- **image_uri**：训练容器映像 URI。在此示例中，使用 *image_uris.retrieve* 指定 SageMaker XGBoost 训练容器 URI。
- **role**：SageMaker 用来代表您执行任务的 IAM 角色（例如，读取训练结果、从 Amazon S3 调用模型构件以及将训练结果写入 Amazon S3）。
- **instance_count 和 instance_type**：用于模型训练的 Amazon EC2 机器学习计算实例的类型和数量。在本实验中，您将使用一个 ml.m5.xlarge 实例，它具有 4个 CPU、16GB 内存、Amazon Elastic Block Store (Amazon EBS) 存储和高网络性能。
- **output_path**：S3 存储桶的路径，SageMaker 在其中存储模型构件和训练结果。
- **sagemaker_session**：会话对象，用于管理与 SageMaker API 操作和训练作业使用的其他 AWS 服务的交互。
- **rules**：Amazon SageMaker Debugger 内置规则列表。在此示例中，create_xgboost_report() 规则创建了一个 XGBoost 报告，用于了解训练进度和结果。

In [ ]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri = container,
    role = role, 
    instance_count = 1, 
    instance_type ='ml.m5.xlarge',
    output_path = output_path,
    sagemaker_session = sagemaker_session,
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report())
    ]
)

### 任务 1.3：配置超参数

超参数直接控制模型的结构、功能和性能。超参数优化允许数据科学家调整模型性能以获得最佳结果。这个过程是机器学习的重要组成部分，选择合适的超参数值对成功至关重要。

您可以通过调用估算器的 *set_hyperparameters* 方法来设置 XGBoost 算法的超参数。

有关 XGBoost 超参数的更多信息，请参阅 [XGBoost 超参数](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)。

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    verbosity = 0,
    objective = 'binary:logistic',
    num_round = 800
)

### 任务 1.4：运行 SageMaker 训练作业

您已经配置了估算器对象和超参数，现在可以开始训练模型了。fit() 方法启动训练脚本。要开始模型训练，请使用训练和验证数据集调用估算器的 fit() 方法。如果您设置“wait=True”，fit() 方法将显示进度日志并等待训练完成。

In [ ]:
xgb_model.fit(
    {
        "train": train_input,
        "validation": validation_input
    },
    wait=True
)

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i>**预期输出**：如果估算器和超参数配置正确，且训练作业正确启动，您应该能看到以下输出：

```plain
************************
**** EXAMPLE OUTPUT ****
************************

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-09-20-09-56-628
2023-08-09 20:09:56 Starting - Starting the training job...
2023-08-09 20:10:19 Starting - Preparing the instances for trainingCreateXgboostReport: InProgress
......
2023-08-09 20:11:21 Downloading - Downloading input data...
2023-08-09 20:11:55 Training - Downloading the training image...
2023-08-09 20:12:20 Training - Training image download completed.Training in progress....
2023-08-09 20:12:56 Uploading - Uploading generated training model...
2023-08-09 20:13:20 Completed - Training job completed
..Training seconds: 107
Billable seconds: 107
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意**：运行训练大约需要 3–4 分钟。

### 任务 1.5：评估模型

训练作业完成后，您可以下载由 SageMaker Debugger 生成的 XGBoost 训练报告。通过 XGBoost 训练报告，您可以深入了解训练的进度和结果，例如关于迭代的损失函数、特征重要性、混淆矩阵、准确率曲线以及训练的其他统计结果。

对于 SageMaker XGBoost 训练作业，使用 Debugger 的“CreateXgboostReport”规则接收训练进度和结果的综合训练报告。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意**：完成报告大约需要 5–10 分钟。在以下代码块中，系统会检查最终报告是否存在，如果不存在，系统会等待 30 秒，将输出文件夹中的任何新文件复制到本地路径，然后再次检查，直到报告可用为止。

In [ ]:
%%capture
the_file = Path("CreateXgboostReport/xgboost_report.ipynb")
notexists = True
while notexists:
    if the_file.is_file():
        notexists = False
    else:
        time.sleep(30)
        rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.job_name + "/rule-output"
        ! aws s3 ls {rule_output_path} --recursive
        ! aws s3 cp {rule_output_path} ./ --recursive

下一个单元格输出中的链接将在 SageMaker Studio 中打开一个新选项卡。要遵循这些指示，请使用以下选项之一：
- **选项 1**：并排查看选项卡。要从 SageMaker Studio 主窗口创建分屏视图，请将 **lab_2.ipynb** 选项卡拖到一边，或者选择 **lab_2.ipynb** 选项卡，然后从工具栏中选择 **File**（文件）和 **New View for Notebook**（为笔记本新建视图）。现在，您可以在浏览 XGBoost 报告时看到相应指示。
- **选项 2**：在 SageMaker Studio 选项卡之间切换，以遵循这些指示。浏览完 XGBoost 报告后，通过选择 **lab_2.ipynb** 选项卡返回至笔记本。

In [ ]:
display("Click link below to view the XGBoost Training notebook", FileLink("CreateXgboostReport/xgboost_report.ipynb"))

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意**：运行此代码后，您应该会看到以下输出：**'Click link below to view the XGBoost Training notebook' <span style="ssb_sm_blue">CreateXgboostReport/xgboost_report.html</span>**（单击下面的链接可查看 XGBoost 训练笔记本 CreateXgboostReport/xgboost_report.html）

要在新选项卡中打开笔记本，请选择链接。笔记本打开后，在 **Set up notebook environment**（设置笔记本环境）窗口中，配置以下各项：

- 对于**Image**（映像），选择 **Data Science 3.0**。
- 对于 **Kernel**（内核），选择 **Python 3**。
- 选择 **Select**（选择）。

在 **xgboost_report.ipynb** 选项卡的顶部，选择 <i aria-hidden="true" class="fas fa-forward"></i> **Restart Kernel and Run All Cells**（重启内核并运行所有单元格）按钮。当出现 **Restart Kernel?**（重启内核？）提示时，选择 **Restart**（重启）

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意**：运行所有单元格大约需要 2–3 分钟。

当所有单元都运行完毕后，向下滚动到 **Confusion Matrix**（混淆矩阵）。混淆矩阵通过比较观察结果的预测类及其真实类，在表中说明了每个类的正确预测数量和不正确预测数量。当您转到图表时，您会看到 **true positive (TP)**（真阳性 (TP)）、**true negative (TN)**（真阴性 (TN)）、**false positive (FP)**（假阳性 (FP)）和 **false negative (FN)**（假阴性 (FN)）值。

- **True positive**（真阳性）：如果实际分类为阳性且预测分类为阳性 (1,1)，则由于阳性样本被分类器正确识别，会将结果视为 **true positive (TP)**（真阳性 (TP)）。
- **False negative**（假阴性）：如果实际分类为阳性而预测分类为阴性 (1,0)，则由于阳性样本被分类器不正确地识别为阴性，会将结果视为 **false negative (FN)**（假阴性 (FN)）。
- ***False positive**（假阳性）：如果实际分类为阴性而预测分类为阳性 (0,1)，则由于阴性样本被分类器不正确地识别为阳性，会将结果视为 **false positive (FP)**（假阳性 (FP)）。
- **True negative**（真阴性）：如果实际分类为阴性且预测分类为阴性 (0,0)，则由于阴性样本被分类器正确识别，会将结果视为 **true negative (TN)**（真阴性 (TN)）。

接下来，向下滚动到 **Evaluation of the Confusion Matrix**（混淆矩阵评估），并仔细查看 **Classification report**（分类报告），以了解每个类别的查准率、查全率和 F1 分数的摘要。

- **Precision**（查准率）：衡量预测为阳性的实际阳性占所有预测为阳性的实际阳性的比例。范围为 0 到 1，值越大准确率越高。查准率表示您的模型认为相关的数据点和实际相关的数据点的比例。查准率是要考虑的一个很好的度量指标，特别是当 FP 的成本很高时。
- **Recall/Sensitivity/True Positive Rate (TPR)**（查全率/敏感度/真阳性率 (TPR)）：衡量被预测为阳性的实际阳性的比例。范围也是 0 到 1，值越大表示预测准确率越高。这也被称为 Recall/Sensitivity（查全率/敏感度）。此度量指标表示在数据集中找到所有相关实例的能力。
- **F1-Score**（F1 分数）：显示您的目标指标，这是查准率和查全率的调和平均值。F1 同时考虑 FP 和 FN，以给予查准率和查全率相同的权重。

您试图预测人们的收入是否低于 50,000 美元，这样您就可以向合格的公民推广政府援助服务。在这种情况下，F1 分数是一个很好的度量标准，因为它同时考虑了 FP（收入超过 50,000 万美元的人被标记为收入低于 50,000 美元）和 FN（收入低于 50,000 美元的人被标记为收入超过 50,000 美元）。您需要确保您的查准率和查全率都很高，F1 分数会同时考虑这两个度量指标。在下一个实验中，您可以通过调整超参数来优化模型，看看是否可以获得更高的 F1 分数。

在本模型中，什么是**查准率**、*查全率**、**F1 分数**和**整体准确率**？

<i aria-hidden="true" class="far fa-comment" style="color:#008296"></i>**思考**：花点时间查看笔记本中包含的其他图表。您看到了哪些类型的信息？ 在您训练自己的模型时，哪些信息会对您有所帮助？

### 任务 1.6：查看模型构件

SageMaker 将模型构件存储在 S3 存储桶中。要找到模型构件的位置，请遵循以下步骤：

1.从左侧菜单栏选择存储桶图标。

1.在存储桶列表中，打开名称中包含 **labdatabucket** 的 Amazon S3 存储桶。

1.导航到 **scripts/data/output/ sagemaker-xgboost-.../output** 子文件夹。

您可以在子文件夹中看到模型构件 **model.tar.gz**。这是您通过调用 fit() 方法使用 SageMaker Estimator 创建的模型。

您已查看模型构件，包括 model.tar.gz 文件。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并继续执行任务 2。